In [1]:
%%capture
!pip install torch
!pip install torchvision
!pip install torchmetrics pytorch-fid
!pip install torchmetrics[image]
!pip install torch-fidelity

In [2]:
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torch import nn
from tqdm import tqdm
from torchsummary import summary
from torchmetrics.image.inception import InceptionScore
from torchmetrics.image.fid import FrechetInceptionDistance
from torchvision import models


In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
training_data = datasets.MNIST(
    root="./data",
    train=True,
    download=True,
    transform=transform
)

test_data = datasets.MNIST(
    root="./data",
    train=False,
    download=True,
    transform=transform
)

train_dataloader = DataLoader(training_data, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=32, shuffle=True)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:00<00:00, 16.0MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 462kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:00<00:00, 4.02MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 7.92MB/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [4]:
print("Number train set: ", len(training_data))
print("Number test set: ", len(test_data))

Number train set:  60000
Number test set:  10000


In [5]:
class FCCGenerator(nn.Module):
    def __init__(self):
        super(FCCGenerator, self).__init__()
        self.fc_layers = nn.Sequential(
            # Input: Z(100)
            nn.Linear(100, 64),  # FC(64)
            nn.ReLU(),           # ReLU

            nn.Linear(64, 512),  # FC(512)
            nn.ReLU(),           # ReLU

            nn.Linear(512, 1152),      # FC(1152)
            nn.BatchNorm1d(1152),      # BN
        )

        self.conv_layers = nn.Sequential(
            # Reshape to (128, 3, 3)
            nn.Unflatten(1, (128, 3, 3)),

            # CONVT(64,3,2), BN, ReLU
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=0, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            # CONVT(32,3,2), BN, ReLU
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(),

            # CONVT(1,3,2), Tanh
            nn.ConvTranspose2d(32, 1, kernel_size=3, stride=2, padding=1, output_padding=1, bias=False),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.fc_layers(x)
        x = self.conv_layers(x)
        return x

class FCCDiscriminator(nn.Module):
    def __init__(self):
        super(FCCDiscriminator, self).__init__()
        self.conv_layers = nn.Sequential(
            # Input: (28, 28, 1)

            # CONV(32,3,2), BN, LReLU
            nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.2),

            # CONV(64,3,2), BN, LReLU
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2),

            # CONV(128,3,2), BN, LReLU
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
        )

        self.fc_layers = nn.Sequential(
            # Flatten (2048)
            nn.Flatten(),

            # FC(512), LReLU
            nn.Linear(2048, 512),
            nn.LeakyReLU(0.2),

            # FC(64), LReLU
            nn.Linear(512, 64),
            nn.LeakyReLU(0.2),

            # FC(16), LReLU
            nn.Linear(64, 16),
            nn.LeakyReLU(0.2),

            # FC(1), Sigmoid
            nn.Linear(16, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        # print("Discriminator Input shape:", x.shape)

        # Pass through convolutional layers
        for layer in self.conv_layers:
            x = layer(x)
            # print(f"{layer.__class__.__name__} output shape: {x.shape}")

        # Pass through fully connected layers
        for layer in self.fc_layers:
            x = layer(x)
            # print(f"{layer.__class__.__name__} output shape: {x.shape}")

        return x






In [6]:
# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
generator = FCCGenerator()
discriminator = FCCDiscriminator()
#Define BCE Loss
bce_loss = nn.BCELoss()

#Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=0.0002)
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=0.0002)


In [8]:
generator.to(device)
discriminator.to(device)


FCCDiscriminator(
  (conv_layers): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.2)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): LeakyReLU(negative_slope=0.2)
  )
  (fc_layers): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=2048, out_features=512, bias=True)
    (2): LeakyReLU(negative_slope=0.2)
    (3): Linear(in_features=512, out_features=64, bias=True)
    (4): LeakyReLU(negative_slope=0.2)
    (5): Linear(in_features=64, 

In [9]:
# Parameters
num_epochs = 20
latent_dim = 100
fixed_noise = torch.randn(64, latent_dim).to(device)

for epoch in range(num_epochs):
    epoch_progress = tqdm(train_dataloader, desc=f"Epoch [{epoch+1}/{num_epochs}]", leave=False)

    for batch_idx, (real_images, _) in enumerate(epoch_progress):
        # TRAIN DISCRIMINATOR
        real_images = real_images.to(device)
        batch_size = real_images.size(0)

        # Label thật và giả
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # Tính loss với ảnh thật
        real_outputs = discriminator(real_images)
        d_loss_real = bce_loss(real_outputs, real_labels)

        # Sinh ảnh giả
        noise = torch.randn(batch_size, latent_dim).to(device)
        fake_images = generator(noise)

        # Tính loss với ảnh giả
        fake_outputs = discriminator(fake_images)
        d_loss_fake = bce_loss(fake_outputs, fake_labels)

        d_loss = d_loss_real + d_loss_fake

        # Cập nhật Discriminator
        optimizer_D.zero_grad()
        d_loss.backward()
        optimizer_D.step()

        # TRAIN GENERATOR
        # Sinh lại ảnh giả
        fake_images = generator(noise)
        # Tính loss của G
        fake_outputs = discriminator(fake_images)
        g_loss = bce_loss(fake_outputs, real_labels)

        # Cập nhật Generator
        optimizer_G.zero_grad()
        g_loss.backward()
        optimizer_G.step()

        # Cập nhật tqdm progress bar
        epoch_progress.set_postfix(D_Loss=f"{d_loss.item():.4f}", G_Loss=f"{g_loss.item():.4f}")



    tqdm.write(f"Epoch [{epoch+1}/{num_epochs}] | D Loss: {d_loss.item():.4f} | G Loss: {g_loss.item():.4f} ")

    # Save generated images for visualization
    with torch.no_grad():
          fake_images = generator(fixed_noise)
          grid = torchvision.utils.make_grid(fake_images, nrow=8, normalize=True)
          plt.figure(figsize=(8, 8))
          plt.imshow(grid.permute(1, 2, 0).cpu().numpy())
          plt.title(f"Epoch {epoch+1}")
          plt.show()



Output hidden; open in https://colab.research.google.com to view.

In [10]:
torch.cuda.empty_cache()


In [12]:
num_epochs = 10
# Initialize FID and Inception Score
fid = FrechetInceptionDistance(feature=2048).to(device) # Default Feature is 2048
inception = InceptionScore().to(device)
for epoch in range(num_epochs):
    with torch.no_grad():
        batch_size = 16
        num_samples = 1000
        for _ in range(num_samples // batch_size):
            noise_eval = torch.randn(batch_size, latent_dim).to(device)
            fake_images = generator(noise_eval)

            # Convert to 3 channels
            fake_images_3channel = fake_images.repeat(1, 3, 1, 1)
            real_images_3channel = real_images[:batch_size].repeat(1, 3, 1, 1).to(device)

            fid.update(real_images_3channel.to(torch.uint8), real=True)
            fid.update(fake_images_3channel.to(torch.uint8), real=False)
            inception.update(fake_images_3channel.to(torch.uint8))

        fid_score = fid.compute()
        inception_score, inception_std = inception.compute()

        print(f"Epoch [{epoch+1}/{num_epochs}] | FID: {fid_score:.4f}")

        fid.reset()
        inception.reset()

Downloading: "https://github.com/toshas/torch-fidelity/releases/download/v0.2.0/weights-inception-2015-12-05-6726825d.pth" to /root/.cache/torch/hub/checkpoints/weights-inception-2015-12-05-6726825d.pth
100%|██████████| 91.2M/91.2M [00:00<00:00, 341MB/s]
/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: Metric `InceptionScore` will save all extracted features in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)  # noqa: B028


Epoch [1/10] | FID: 227.2533
Epoch [2/10] | FID: 227.0006
Epoch [3/10] | FID: 226.6088
Epoch [4/10] | FID: 226.7691
Epoch [5/10] | FID: 226.4233
Epoch [6/10] | FID: 228.8232
Epoch [7/10] | FID: 227.4724
Epoch [8/10] | FID: 225.6919
Epoch [9/10] | FID: 227.7406
Epoch [10/10] | FID: 228.4251
